Задача по извлечению фич из ЭМГ.
Основная статья
https://www.researchgate.net/publication/325226027_Feature_Extraction_and_Selection_for_Myoelectric_Control_Based_on_Wearable_EMG_Sensors

1. Фичи, харктеризующие мощность и амплитуду сигнала

    Integrated Absolute Value IAV
    
    Mean Absolute Value MAV
    
    #### Root Mean Square RMS
    
    #### Variance VAR
    
    #### Waveform Length WL
    
    Log Detector LD
    
    #### Difference Absolute Mean Value DAMV
    
    #### Differebce Absolute Standard Deviation Value DASDV
    
    #### Differebce Variance Value DVARV
    
    #### Mean Value of Square Root MSR
    
    #### L-scale LS 

2. Фичи, характеризующие нелинейность и частотную информацию

    #### Maximum Fractal Length MFL
    
    https://www.mdpi.com/2079-3197/7/1/12/htm
    
    Detrended fluctuation analysis DFA
    
    Sample Entropy SampEn
    
    #### Zero Crossing ZC
    
    Slope Sign Change SSC
    
    #### Willison Amplitude WAMP
    
    Median Frequency MDF
    
    Mean Frequency MNF
    
3. Фичи, использующие ЭМГ как временные ряды

    Autoregressive Coefficients AR
    
    Cepstrum Coefficients CC
    
    Разного порядка 4 6 9
   
4. Уникальные

    HIST - расширение ZC и WAMP 9 data bins
    
Предложенные группы фичей
1. MAV WL ZC SSC
2. RMS AR6
3. MAV WL ZC SSC RMS AR6
4. AR4 HIST9
5. WL LD SSC AR9
6. WL SSC AR9 CC9
7. RMS VAR LD HIST9
8. WL RMS SampEn CC4

Предложенные комбинации для myo (через forward feature selection)
1. LS + MFL + MSR + WAMP 
2. + ZC + RMS + IAV + DASDV + VAR 

Частота сэмплов 200 Гц и 1000 Гц
SVM linear kernel

Выводы
Для данных с разной частотой производительность фич отличается.
Хорошо работают фичи первой группы. 
Фичи третьей группы лучше для частоты 1000 Гц

Датасеты ninaPro

1. 52 движения, 27 человек. Нужны столбцы 12 emg, stimulus (что пытались повторить), restimulus(что в итоге повторили). Otto Bock. Повторение движения 10 раз. 100 Гц

2. 40 человек, Delsys, 49 движений, повторение 6 раз 2000 Гц
3. То же, что и 2, но 11 инвалидов 2000 Гц

4. Cometa 10 человек 2000 Гц
5. Myo 10 человек 200 Гц

6. 10 человек в течение 5 дней. Delsys Для изучения воспроизводимости 2000 Гц

7. 20 человек 2 инвалида 40 классов Delsys 2000 Гц

In [37]:
#Загрузим несколько файлов датасета
import scipy.io
data1 = scipy.io.loadmat('DB2_s4/S4_E1_A1.mat')
data2 = scipy.io.loadmat('DB2_s4/S4_E2_A1.mat')
data3 = scipy.io.loadmat('DB2_s4/S4_E3_A1.mat')

#Файлы E1 E2 E3 - классы упражнений

In [72]:
def from_mat_to_numpy(data):
    """
    Функция для формирования датасета в виде, который можно скормить классификатору.
    
    data - словарь, полученный в результате чтения .mat файла. Ключи в нём - названия, значения - значения в виде numpy массивов.
    Интересующие нас:
        emg - сигнал
        stimulus - номер выполняемого движения
        repetition - номер повтора движения
        restimulus - номер движения по факту
        
    Сигнал, соотв. одному и тому же повторению и движению, склеиваем в один. И сопоставляем соотв.номер движения и повторения.
    Пропускаем моменты отдыха (движение 0).
    """
    
    emg = data1['emg']
    target = data1['stimulus']
    repetition = data1['repetition']
    restimulus = data1['restimulus']
    
    n = emg.shape[0]
    
    curr_target = 0
    curr_rep = 0
    current_emg = []
    preprocess_data = [[] for i in range(17*6)]
    index = 0
    for i in range(n):
        if target[i][0] == curr_target and repetition[i][0] == curr_rep:
            if curr_target != 0 and curr_rep != 0:
                current_emg.extend(emg[i])
        else:
            if curr_target != 0 and curr_rep != 0:
                current_emg.append(curr_target)
                current_emg.append(curr_rep)
                preprocess_data[index] = current_emg
                index += 1
            curr_target = target[i][0]
            curr_rep = repetition[i][0]
            current_emg = list(emg[i])
    return preprocess_data

In [73]:
new_data = from_mat_to_numpy(data1)

In [85]:
print(len(new_data))
print(len(new_data[0]))
print(len(new_data[1]))

102
139418
131294


In [3]:
import numpy as np

In [77]:
class feature_exctraction:
    """
    Feature extraction from EMG signal class.
    Fields: 
        rate - rate of emg signal in Hz, 
        window_size - width of window in ms, from which one value of new feature will be computed
        overlap - in ms, overlap of windows
    """
    def __init__(self, rate, window_size, overlap):
        self.rate = rate
        self.window_len = int(self.rate/1000*window_size)  # Соотв. длина окна
        self.overlap_len = int(self.rate/1000*overlap)  # Соотв. длина пересечения
        
    def integrated_absolute_value(self, emg):
        """
        Compute integrated absolute value of emg signal.
        Parameters:
            emg - numpy array of size (number_of_samples, length_of_signal)
        Return:
            numpy array with new feature
        """
        number_of_samples, signal_len = emg.shape
        X = [[] for _ in range(number_of_samples)]
        for j in range(number_of_samples):
            # Считаем количество сегментов для вычисления фичи
            n = (signal_len - self.window_len)//(self.window_len - self.overlap_len) + 1
            IAV_feature = []
            # Для каждого сегмента вычисляем нужное значение и записываем его в вектор
            for i in range(n):
                start = i*(self.window_len - self.overlap_len)
                segment = emg[j][start:start+self.window_len]
                IAV_feature.append(sum([abs(s) for s in segment]))
            X[j] = IAV_feature
        return np.array(X)
    
    def mean_absolute_value(self, emg):
        """
        Compute mean absolute value of emg signal.
        Parameters:
            emg - numpy array of size (number_of_samples, length_of_signal)
        Return:
            numpy array with new feature
        """
        number_of_samples, signal_len = emg.shape
        X = [[] for _ in range(number_of_samples)]
        for j in range(number_of_samples):
            # Считаем количество сегментов для вычисления фичи
            n = (signal_len - self.window_len)//(self.window_len - self.overlap_len) + 1
            MAV_feature = []
            # Для каждого сегмента вычисляем нужное значение и записываем его в вектор
            for i in range(n):
                start = i*(self.window_len - self.overlap_len)
                segment = emg[j][start:start+self.window_len]
                MAV_feature.append(sum([abs(s) for s in segment])/len(segment))
            X[j] = MAV_feature
        return np.array(X)
    
    def integrated_absolute_value(self, emg):
        """
        Compute integrated absolute value of emg signal.
        Parameters:
            emg - numpy array of size (number_of_samples, length_of_signal)
        Return:
            numpy array with new feature
        """
        number_of_samples, signal_len = emg.shape
        X = [[] for _ in range(number_of_samples)]
        for j in range(number_of_samples):
            # Считаем количество сегментов для вычисления фичи
            n = (signal_len - self.window_len)//(self.window_len - self.overlap_len) + 1
            IAV_feature = []
            # Для каждого сегмента вычисляем нужное значение и записываем его в вектор
            for i in range(n):
                start = i*(self.window_len - self.overlap_len)
                segment = emg[j][start:start+self.window_len]
                IAV_feature.append(sum([abs(s) for s in segment]))
            X[j] = IAV_feature
        return np.array(X)

In [80]:
#Test
emg_1 = np.array([[-1, -2, 4, 5, -3, 7, -5, 4, 1, 2]])
X = feature_exctraction(1000, 2, 1).integrated_absolute_value(emg_1)
Y = feature_exctraction(1000, 2, 1).mean_absolute_value(emg_1)
print(X)
print(Y)

[[ 3  6  9  8 10 12  9  5  3]]
[[ 1.5  3.   4.5  4.   5.   6.   4.5  2.5  1.5]]
